In [1]:
!pip install pymysql

    100% |████████████████████████████████| 81kB 458kB/s ta 0:00:01


In [2]:
import pymysql as mariadb

try:
    conn = mariadb.connect(
     host='db',
     user='root',
     passwd='1234',
     db='rental',
     charset='utf8mb4'
    )
    
    cursor = conn.cursor()
    sql = """
        CREATE TABLE `baseEnviro` (
  `ID` INT NOT NULL AUTO_INCREMENT,
  `url` VARCHAR(100) NULL,
  `name` VARCHAR(100) NULL,
  `address` VARCHAR(200) NULL,
  `typeID` INT NULL,
  `cost` INT NULL,
  `description` TEXT(2000) NULL,
  `lat` DECIMAL(20,17) NULL,
  `lng` DECIMAL(20,17) NULL,
  `cityID` INT NULL,
  PRIMARY KEY (`ID`),
  UNIQUE INDEX `url_UNIQUE` (`url` ASC))
  ENGINE = InnoDB DEFAULT  
  CHARACTER SET = utf8mb4 COLLATE = utf8mb4_general_ci;
        """
    try:
        cursor.execute(sql)
    except Exception as e:
        print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
        cursor.close()
        conn.close()
except Exception as e:
    print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

In [1]:
import glob
from multiprocessing import Process, Queue
import json
q1=Queue()
fil = glob.glob('Food/supermarket/*')
for detail in fil:
    with open('{}'.format(detail),encoding='utf-8') as f:
        jSon = json.load(f)
        #print(jSon)
        q1.put(jSon)

print(q1.qsize())

389


In [2]:
def write(lock,obj):
    obj = obj
    lock.acquire()
    try:
        conn = mariadb.connect(
            host='db',
            user='root',
            passwd='1234',
            db='rental',
            charset='utf8mb4'
        )
        cursor = conn.cursor()
        sql = '''INSERT INTO baseEnviro (url,name,address,typeID,cost,description,lat,lng,cityID) VALUES 
                (%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
        values = (obj['url'], obj['name'], obj['address'],obj['typeID'],obj['cost'],
                  obj['description'], obj['lat'],obj['lng'], obj['cityID'])
        try:
            cursor.execute(sql,values)
            conn.commit()
        except Exception as e:
            print(obj)
            conn.rollback()
            print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

        cursor.close()

        conn.close()
    except Exception as e:
        print(obj)
        print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
    finally:
        lock.release()

In [ ]:
import pymysql as mariadb
import time
from multiprocessing import Process, Lock
lock = Lock()
start = time.time()
while not q1.empty():
    try:
        obj = q1.get()
        p = Process(target=write, args=(lock, obj)) 
        p.start()
        p.join()
    except Exception as e:
        print(e)
        print(obj)
print('total time = {}'.format(time.time()-start))

In [ ]:
# Metadata



## ipeen Total:34493 -> sucess:34493 , fail:0
## spending time:1099sec

## supermar